In [273]:
import pandas as pd
import numpy as np
import re

In [274]:
# pd.set_option('display.max_colwidth', None)

In [275]:
pd.reset_option('display.max_colwidth')

In [276]:
master_data = {
    "otodom_2023_master" : "../original_data_2023/otodom/ALL_DATA/gdansk_otodom_master_data.json",
}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

otodom_df = master_data["otodom_2023_master"]

In [277]:
df = otodom_df

In [278]:
df.shape

(4535, 12)

In [279]:
df.head()

,link,address,title,price,surface,rent_extra,rooms,deposit,floor,building_type,avilable_since,outside_are
0,https://www.otodom.pl/pl/oferta/mieszkanie-z-w...,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",Mieszkanie z widokiem na Motławę!,2 500 zł,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,None,2023-08-14,None
1,https://www.otodom.pl/pl/oferta/dwa-pokoje-ant...,"Oliwa, Gdańsk, pomorskie","Dwa pokoje+antresola, niezależna kuchnia/jadalnia",3 300 zł,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący,None,None
2,https://www.otodom.pl/pl/oferta/pacific-wynaje...,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",Pacific | wynajem | balkon | parking,3 600 zł,44 m²,860 zł/miesiąc,2.0,3 600 zł,None,None,None,None
3,https://www.otodom.pl/pl/oferta/apartament-2-p...,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",Apartament 2-pokojowy prestiżowe osiedle River...,4 100 zł,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec,None,None
4,https://www.otodom.pl/pl/oferta/mieszkanie-z-b...,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",Mieszkanie z balkonem | Świetna komunikacja,3 200 zł,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok,None,balkon


In [280]:
otodom_gda_districts_mapping = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    "Przymorze Wielkie": "Przymorze",
    "Przymorze Małe": "Przymorze", 
    "VII Dwór": "Przymorze",
    
    "Południe": "Chelm - Lostowice - Poludnie",
    "Łostowice": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
    
    "Orunia Górna - Gdańsk Południe": "Chelm - Lostowice - Poludnie",
    "Ujeścisko-Łostowice ": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
    
    "Wrzeszcz Górny": "Wrzeszcz",
    "Wrzeszcz Dolny": "Wrzeszcz",
    
    "Brzeźno": "Brzezno",
    
    "Jelitkowo": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Żabianka": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Wejhera": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Tysiąclecia": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",

    
    "Siedlce": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Młyniska": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Wzgórze Mickiewicza": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Osowa": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    
    "Zaspa-Rozstaje": "Zaspa",
    "Zaspa-Młyniec": "Zaspa",
    
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Brętowo": "Piecki Migowo - Suchanino - Morena", 
    "Piecki-Migowo": "Piecki Migowo - Suchanino - Morena",
    
    "Letnica": "Letnica",
    
    "Oliwa": "Oliwa - Strzyza",
    "Strzyża": "Oliwa - Strzyza",
    
    "Orunia-Św. Wojciech-Lipce": "Orunia",
    "Krakowiec-Górki Zachodnie": "Orunia",
    "Olszynka": "Orunia",
    
    "Nowy Port": "Przerobka - Stogi - Nowy Port",
    "Przeróbka": "Przerobka - Stogi - Nowy Port",
    "Stogi": "Przerobka - Stogi - Nowy Port",
    "Wyspa Sobieszewska": "Przerobka - Stogi - Nowy Port",
}

otodom_rooms_mapping = {
    3: 3,
    2: 2,
    1: 1,
    4: 4,
    5: 4,
    6: 4,
    7: 4
}

# otodom_level_mapping = {
#     'parter': 0,
#     '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10,
#     '11': 11,
#     '12': 11,
#     '13': 11,
#     '14': 11,
#     '15': 11,
#     '16': 11,
#     '17': 11,
#     'poddasze': 11,
# }

otodom_building_type_mapping = {
    "apartamentowiec": "apartment_building",
    "blok": "unit",
    "Unknown": "unknown",
    "kamienica": "tenement",
    "dom wolnostojący": "house",
    "szeregowiec": "house",
    "plomba": "unit",
}

In [281]:
dirty_data = df.copy()

In [282]:
df.isna().sum()

link                 0
address             43
title               43
price               43
surface             31
rent_extra        1302
rooms               31
deposit           1570
floor              228
building_type      921
avilable_since    3355
outside_are       1439
dtype: int64

In [283]:
df.columns

Index(['link', 'address', 'title', 'price', 'surface', 'rent_extra', 'rooms',
       'deposit', 'floor', 'building_type', 'avilable_since', 'outside_are'],
      dtype='object')

In [284]:
df.head()

,link,address,title,price,surface,rent_extra,rooms,deposit,floor,building_type,avilable_since,outside_are
0,https://www.otodom.pl/pl/oferta/mieszkanie-z-w...,"ul. Żabi Kruk, Śródmieście, Gdańsk, pomorskie",Mieszkanie z widokiem na Motławę!,2 500 zł,"39,27 m²",500 zł/miesiąc,2.0,3 000 zł,parter/8,None,2023-08-14,None
1,https://www.otodom.pl/pl/oferta/dwa-pokoje-ant...,"Oliwa, Gdańsk, pomorskie","Dwa pokoje+antresola, niezależna kuchnia/jadalnia",3 300 zł,65 m²,650 zł/miesiąc,2.0,3 300 zł,2/2,dom wolnostojący,None,None
2,https://www.otodom.pl/pl/oferta/pacific-wynaje...,"ul. Obrońców Wybrzeża 15, Przymorze Wielkie, G...",Pacific | wynajem | balkon | parking,3 600 zł,44 m²,860 zł/miesiąc,2.0,3 600 zł,None,None,None,None
3,https://www.otodom.pl/pl/oferta/apartament-2-p...,"ul. Angielska Grobla, Śródmieście, Gdańsk, pom...",Apartament 2-pokojowy prestiżowe osiedle River...,4 100 zł,"53,2 m²",934 zł/miesiąc,2.0,5 500 zł,5/5,apartamentowiec,None,None
4,https://www.otodom.pl/pl/oferta/mieszkanie-z-b...,"ul. Piotrkowska, Ujeścisko-Łostowice, Gdańsk, ...",Mieszkanie z balkonem | Świetna komunikacja,3 200 zł,"55,5 m²",700 zł/miesiąc,3.0,6 400 zł,3/3,blok,None,balkon


In [285]:
df_dirty = dirty_data.copy()

In [395]:
import re

def convert_numeric_to_float(convertable_string):
    return float(convertable_string
                 .replace("zł", "").replace(" ", "").replace("\ndonegocjacji", "")
                 .replace(",",".").replace("/miesiąc", "").replace("m²", ""))
    
def extract_district(address):
    # Wyrażenie regularne, które szuka nazwy dzielnicy, niezależnie od tego, czy zaczyna się od "ul." czy nie
    pattern = re.compile(r'(?:ul\. )?([^,]+), Gdańsk, pomorskie')
    
    match = pattern.search(address)
    if match:
        return match.group(1).strip()
    else:
        return None
    
def convert_floor_to_level(floor):
    floor = str(floor).replace("suterena", "0").replace("poddasze", "11")
    if 'parter' in str(floor).lower():
        return 0
    elif ">" in str(floor):
        return 11
    elif '/' in str(floor):
        try:
            # Extract numeric part and convert to int
            numeric_part = ''.join(char for char in floor.split('/')[0] if char.isdigit() or char == '-')
            return int(numeric_part)
        except ValueError:
            print(floor)
        return None
    elif str(floor).isdigit():
        return int(floor)
    else:
        return None
    
def clear_otodom(df: pd.DataFrame):
    
    df.drop(
        labels=[
            'deposit', 
        ],
        inplace= True,
        axis= 1
    )
    
    df.dropna(subset=['address', 'price', 'surface', 'rooms'], inplace=True)
    
    df['floor'] = df['floor'].fillna(1001)
    df["building_type"] = df["building_type"].fillna("Unknown")
    
    df.drop(
        labels=[
            'avilable_since',
            'outside_are',
            'link',
            'title'
        ],
        inplace= True,
        axis= 1
    )
    
    # rent
    df = df.rename(mapper={"price": "rent"}, axis=1)
    df = df[~df['rent'].str.contains("EUR", na=False)]
    df['rent'] = df['rent'].apply(convert_numeric_to_float)

    
    # building_type
    df["building_type"] = df["building_type"].map(otodom_building_type_mapping)
    
    # level
    df = df.rename(mapper={"floor": "level"}, axis=1)
    df["level"] = df["level"].apply(convert_floor_to_level)
    df.dropna(subset=['level'], inplace=True)
    
    # surface
    df["surface"] = df["surface"].apply(convert_numeric_to_float)
    
    
    # rent_extra
    df["rent_extra"] = df["rent_extra"].fillna("0")
    df['rent_extra'] = df['rent_extra'].apply(convert_numeric_to_float)

    # location
    df = df.rename(mapper={"address": "location"}, axis=1)
    df["location"] = df["location"].apply(extract_district)
    df["location"] = df["location"].map(otodom_gda_districts_mapping)
    df.dropna(subset=['location'], inplace=True)
    
    #rooms
    df["rooms"] = df["rooms"].map(otodom_rooms_mapping)
    
    return df

In [398]:
df_dirty = dirty_data.copy()

In [399]:
df = clear_otodom(df_dirty)

In [365]:
df.head()

,location,rent,surface,rent_extra,rooms,level,building_type
0,Srodmiescie - Stare Miasto - Dolne Miasto - Dł...,2500.0,39.27,500.0,2,0.0,unknown
1,Oliwa - Strzyza,3300.0,65.00,650.0,2,2.0,house
2,Przymorze,3600.0,44.00,860.0,2,1001.0,unknown
3,Srodmiescie - Stare Miasto - Dolne Miasto - Dł...,4100.0,53.20,934.0,2,5.0,apartment_building
5,Siedlce - Aniolki - Mlyniska - Mickiewicza,2600.0,70.00,1005.0,3,2.0,unit
